<a href="https://colab.research.google.com/github/PsorTheDoctor/computer-vision/blob/master/point_cloud_processing/pose_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Point cloud pose estimation

In [ ]:
%pip install -q open3d

In [152]:
import open3d as o3d
import numpy as np
import copy
from tqdm import tqdm
import random

scene = o3d.io.read_point_cloud('scene.pcd')
obj_mesh = o3d.io.read_triangle_mesh('duck.stl')
obj = obj_mesh.sample_points_poisson_disk(10000)

In [153]:
def remove_downward_normals(pcd):
  """
  Remove points whose normals are pointed downward.
  """
  pcd.estimate_normals()
  normals = np.asarray(pcd.normals)
  threshold = -0.5
  indices = np.where(normals[:, 2] >= threshold)[0]
  filtered_pcd = pcd.select_by_index(indices)
  print(f'{10000 - len(indices)} dropped out.')
  return filtered_pcd

def visualize(scene, obj, transformation=np.eye(4)):
  scene_temp = copy.deepcopy(scene)
  obj_temp = copy.deepcopy(obj)
  scene_temp.paint_uniform_color([1, 0.706, 0])
  obj_temp.paint_uniform_color([0, 0.651, 0.929])
  obj_temp.transform(transformation)
  o3d.visualization.draw_plotly([scene_temp, obj_temp])

##Preprocessing

In [154]:
hollow_obj = remove_downward_normals(obj)
o3d.visualization.draw_plotly([hollow_obj])

2820 dropped out.


##Global estimation: RANSAC

In [155]:
def global_estimation(scene, obj):
  tree = o3d.geometry.KDTreeSearchParamKNN(knn=10)
  obj.estimate_normals(tree)
  scene.estimate_normals(tree)

  tree2 = o3d.geometry.KDTreeSearchParamRadius(0.05)
  obj_features = o3d.pipelines.registration.compute_fpfh_feature(obj, tree2)
  scene_features = o3d.pipelines.registration.compute_fpfh_feature(scene, tree2)

  obj_features = np.asarray(obj_features.data).T
  scene_features = np.asarray(scene_features.data).T
  corr = o3d.utility.Vector2iVector()

  for j in tqdm(range(obj_features.shape[0]), desc='Correspondences'):
    fobj = obj_features[j]
    dist = np.sum((fobj - scene_features) ** 2, axis=-1)
    kmin = np.argmin(dist)
    corr.append((j, kmin))

  tree3 = o3d.geometry.KDTreeFlann(scene)
  iters = 500
  threshold = 0.0001
  random.seed(123456789)
  inliers_best = 0

  for _ in tqdm(range(iters), desc='RANSAC'):
    corr_ = o3d.utility.Vector2iVector(random.choices(corr, k=3))
    est = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    T = est.compute_transformation(obj, scene, corr_)

    obj_aligned = o3d.geometry.PointCloud(obj)
    obj_aligned.transform(T)
    inliers = 0

    for j in range(len(obj_aligned.points)):
      k, idx, dist = tree3.search_knn_vector_3d(obj_aligned.points[j], 1)
      if dist and dist[0] < threshold:
        inliers += 1

    if inliers > inliers_best:
      print(f'Got a new model with {inliers}/{len(obj_aligned.points)} inliers!')
      inliers_best = inliers
      pose = T

  print('Global pose estimation:', pose)
  return pose

pose = global_estimation(scene, hollow_obj)
visualize(scene, hollow_obj, pose)

RANSAC:   0%|          | 2/500 [00:00<00:51,  9.74it/s]

Got a new model with 2067/7180 inliers!


RANSAC:   1%|          | 6/500 [00:00<00:51,  9.59it/s]

Got a new model with 2255/7180 inliers!


RANSAC:   2%|▏         | 10/500 [00:01<00:53,  9.24it/s]

...

Got a new model with 4060/7180 inliers!


RANSAC:  42%|████▏     | 210/500 [00:18<00:20, 14.09it/s]

Got a new model with 6195/7180 inliers!


RANSAC: 100%|██████████| 500/500 [00:44<00:00, 11.29it/s]


Global pose estimation: [[-0.60232801 -0.78649294 -0.13649108  0.01565242]
 [ 0.73781291 -0.4832618  -0.47126442  0.12904791]
 [ 0.30468521 -0.38456064  0.87136676 -0.64928817]
 [ 0.          0.          0.          1.        ]]


##Local estimation: Robust ICP

In [156]:
def local_estimation(scene, obj, init_pose=None, robust=True):
  tree = o3d.geometry.KDTreeFlann(scene)
  iters = 50
  thresh_squared = 0.0001
  pose = init_pose
  obj_aligned = o3d.geometry.PointCloud(obj)
  loss = o3d.pipelines.registration.TukeyLoss(k=0.1) if robust else None

  for _ in tqdm(range(iters), desc='ICP'):
    corr = o3d.utility.Vector2iVector()
    for j in range(len(obj_aligned.points)):
      k, idx, dist = tree.search_knn_vector_3d(obj_aligned.points[j], 1)
      if dist and dist[0] < thresh_squared:
        corr.append((j, idx[0]))

    est = o3d.pipelines.registration.TransformationEstimationPointToPlane(loss)
    T = est.compute_transformation(obj_aligned, scene, corr)
    obj_aligned.transform(T)
    pose = T if pose is None else T @ pose

  print('Local pose estimation:', pose)
  return pose

hollow_obj = hollow_obj.transform(pose)
refined_pose = local_estimation(scene, hollow_obj)
visualize(scene, hollow_obj, refined_pose)

ICP: 100%|██████████| 50/50 [00:04<00:00, 12.49it/s]


Local pose estimation: [[ 0.96870642 -0.24813915  0.00590147  0.03622577]
 [ 0.2467372   0.96527452  0.08582457  0.0603871 ]
 [-0.02699297 -0.0816827   0.99629279  0.00705225]
 [ 0.          0.          0.          1.        ]]
